Uppdrag 1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px 

# read in the file, found a lot of NaN
df_file = './Data/riket2023_åk9_np.xlsx'

# reads all sheets into a sictionary of DataFrames
# got information on https://www.geeksforgeeks.org/how-to-read-excel-multiple-sheets-in-python-pandas/
df_sheets = pd.read_excel(df_file, sheet_name=None)
sheets = ['Engelska', 'Matematik', 'Svenska', 'Svenska som andraspråk']

# a dictionary to save the cleaned sheets
cleaned_data = {}

df_sheets["Engelska"].head(10)

In [155]:
# I think I made a mistake in reading in the sheets, cause row 7 kept coming up
# and I couldn't use skiprows cause then the columns did not read in correctly
# so I decided to remove it instead
for sheet in sheets:
    df_sheets[sheet] = df_sheets[sheet].drop(index=7)

In [156]:
# a function to rename all the columns in every sheet
def rename_all_columns(df):
    return df.rename(columns={
        'Unnamed: 0': 'Plats', 
        'Unnamed: 1': 'Huvudman', 
        'Unnamed: 2': 'Totalt (A-F)', 
        'Unnamed: 3': 'Flickor (A-F)', 
        'Unnamed: 4': 'Pojkar (A-F)', 
        'Unnamed: 5': 'Totalt (A-E)', 
        'Unnamed: 6': 'Flickor (A-E)', 
        'Unnamed: 7': 'Pojkar (A-E)', 
        'Unnamed: 8': 'Totalt (Poäng)', 
        'Unnamed: 9': 'Flickor (Poäng)', 
        'Unnamed: 10': 'Pojkar (Poäng)'
    })

In [ ]:
# rename the sheets with the function
# using a for loop to go through all sheets
for s in sheets:
    sheet = df_sheets[s]
    sheet = rename_all_columns(sheet)
    cleaned_data[s] = sheet

cleaned_data["Engelska"].info()

In [158]:
# remove the NaNs from each sheet
for d in sheets:
    cleaned_data[d] = cleaned_data[d].dropna()

In [ ]:
# group huvudman and totalt (poäng) to use sum to calculate the total sum of poäng
# first a dictionary to save the grouped data
group_sheets = {}

for g in sheets:
    group_sheets[g] = cleaned_data[g].groupby(["Huvudman"]).agg({"Totalt (Poäng)": "sum"}).reset_index()

group_sheets["Svenska som andraspråk"].head()

In [160]:
# get the DataFrame from each subject from the dictionary
df_english = group_sheets["Engelska"] 
df_math = group_sheets["Matematik"]
df_swedish = group_sheets["Svenska"] 
df_swedish_two = group_sheets["Svenska som andraspråk"]

In [ ]:
# since "Svenska som andraspråk" only contains two ".." I need to convert to NaN
# with replace
df_swedish_two["Totalt (Poäng)"] = df_swedish_two["Totalt (Poäng)"].replace("..", np.nan)

In [162]:
# tried to create different colours for every bar in the chart
# found info on https://python-graph-gallery.com/3-control-color-of-barplots/

# create a function so that I can assign teh colors
def bar_colour(subjects_df):
    colour_huvudman = {
    'Skolverket' : 'purple',
    'Kommunal': 'green',
    'Enskild': 'blue',
    'Samtliga': 'orange'
}
    return[colour_huvudman.get(huvudman, 'black') for huvudman in subjects_df["Huvudman"]]

In [ ]:
# plot the barchart, need to use subplots 
# categorys = the different subjects for the total points in the total subjects
# english, swedish, maths and swedish as second language
fig, axs = plt.subplots(1, 4, dpi = 120, figsize =(15, 4))

# main title
fig.suptitle("Totala poäng för huvudmännen", fontsize=15) 

# first plot, english
axs[0].bar(df_english['Huvudman'], 
    df_english['Totalt (Poäng)'],
    color = bar_colour(df_english)
)
axs[0].set_title("Engelska", fontsize = 10)
axs[0].set_xticks([0,1,2,3])
axs[0].tick_params(axis='x', rotation=90)

# second plot, math
axs[1].bar(df_math['Huvudman'], 
    df_math['Totalt (Poäng)'],
    color = bar_colour(df_math)
)
axs[1].set_title("Matematik", fontsize = 10)
axs[1].set_xticks([0,1,2,3])
axs[1].tick_params(axis='x', rotation=90)

# third plot for swedish
axs[2].bar(df_swedish['Huvudman'], 
    df_swedish['Totalt (Poäng)'],
    color = bar_colour(df_swedish)
)
axs[2].set_title("Svenska", fontsize = 10)
axs[2].set_xticks([0,1,2,3])
axs[2].tick_params(axis='x', rotation=90)

# fourth plot for swedish as second language
axs[3].bar(df_swedish_two['Huvudman'], 
    df_swedish_two['Totalt (Poäng)'].fillna(0),
    color = bar_colour(df_swedish_two)
)
axs[3].set_title("Svenska som andraspråk", fontsize = 10)
axs[3].set_xticks([0,1,2,3])
axs[3].tick_params(axis='x', rotation=90)

plt.show()

Uppgift 2

In [3]:
# read in the specific Tabell from the excel file
# lots of NaN values
df_tabell_sheet = pd.read_excel("./Data/betyg_o_prov_riksnivå.xlsx", sheet_name="Tabell 1B")

In [ ]:
# let's rename the columns first
df_tabell_sheet = df_tabell_sheet.rename(columns ={
        'Unnamed: 1': 'Totalt (16 Ämnen)', 
        'Unnamed: 2': 'Flickor (16 Ämnen)', 
        'Unnamed: 3': 'Pojkar (16 Ämnen)', 
        'Unnamed: 4': 'Totalt (17 Ämnen)', 
        'Unnamed: 5': 'Flickor (17 Ämnen)', 
        'Unnamed: 6': 'Pojkar (17 Ämnen)', 
        'Unnamed: 7': 'Total (%)', 
        'Unnamed: 8': 'Flickor (%)', 
        'Unnamed: 9': 'Pojkar (%)', 
    })

df_tabell_sheet.info()

In [5]:
# need to add Läsår to the column list
df_tabell_sheet["Läsår"] = df_tabell_sheet.iloc[7:13, 0].astype(str)

In [ ]:
# clean the data
df_cleaned_data = df_tabell_sheet.dropna()

df_cleaned_data.isnull().sum()

In [ ]:
# plotly line diagram for students with no passing grade
# found some fun plotting types on https://plotly.com/python/plotly-express/
fig = px.line(df_cleaned_data , x = "Läsår", y = ["Total (%)", "Flickor (%)", "Pojkar (%)"], 
title = "Andel elever som saknar godkänt betyg",
labels = {"value": "Andel (%)", "variable": "",}, line_shape="spline", render_mode="svg", markers = True)
fig.show() 

In [ ]:
  # plotly diagram for meritvärde för 16 ämnen, for pojkar, flickor och totalt
fig = px.line(df_cleaned_data , x = "Läsår", y = ["Totalt (16 Ämnen)", "Flickor (16 Ämnen)", "Pojkar (16 Ämnen)"], 
title = "Meritvärde för 16 ämnen",
labels = {"value": "Ämnen", "variable": "",}, line_shape="spline", render_mode="svg", markers = True)
fig.show()